# **DO NOT HARDCODE NUMBERS or LINKS**
Add any number/link/constant to parameter list below. If see any hardcode number move it to parameter list.

Dataset Google drive folder: https://drive.google.com/open?id=1HXG9ojRsfo8X9w6w4LbfjPJKxJF3UinW
(80% training, 20% testing)

Keeping track of net evolution and attempts Google doc: https://docs.google.com/document/d/19fguUEgDDC_anrR7M10gv2EteymBbQM-lQrwZ94wabs/edit?usp=sharing


# **TODO/CONCERNS**


1.   Implement a better network architecture (now it's just 2 conv, 3 fully connected layers with random nodes for each layer).
2.   If cannot improve the network with 6 category, pick 1 category and clumps the other ones together so it's a binary sorting (ie. metal vs everything, or glass vs everything, etc). 
3.   (For future) Figure out how to save the net so when we get to a good architecture we can keep using it instead of training it over and over again



In [0]:
import numpy as np
import math
import matplotlib.pyplot as plt
import cv2
import torch
import torchvision.transforms as transforms
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# PARAMETER LIST
trainRootDir = "/content/gdrive/My Drive/trash5/train"
testRootDir = "/content/gdrive/My Drive/trash5/test"
train2RootDir = "/content/gdrive/My Drive/trash5/train2"

imageCropSize = 32

trainBatchNum = 32
testBatchNum = 100

epochNum = 50
learnRate = 0.005
momentumNum = 0.9

In [0]:
data_transforms = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
data_transform_augmented = transforms.Compose([
                                    transforms.RandomApply([transforms.RandomVerticalFlip(),
                                                            transforms.RandomHorizontalFlip(),
                                                            transforms.RandomGrayscale(),]),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [0]:
# Training data
train_datasets = torchvision.datasets.ImageFolder(root = trainRootDir, transform = data_transforms)
train_datasets_aug = torchvision.datasets.ImageFolder(root = train2RootDir, transform = data_transform_augmented)
trainloader = torch.utils.data.DataLoader(torch.utils.data.ConcatDataset([train_datasets, train_datasets_aug]), 
                                          batch_size = trainBatchNum, 
                                          shuffle = True, num_workers = 2)
# trainloader = torch.utils.data.DataLoader(train_datasets, batch_size = trainBatchNum, shuffle = True, num_workers = 2)

# Validation data
# valid_datasets = torchvision.datasets.ImageFolder(root = validRootDir, transform = data_transforms)
# validloader = torch.utils.data.DataLoader(valid_datasets, batch_size = trainBatchNum, shuffle = True, num_workers = 2)

# Testing data
test_datasets = torchvision.datasets.ImageFolder(root= testRootDir, transform = data_transforms)
testloader = torch.utils.data.DataLoader(test_datasets, batch_size = testBatchNum, shuffle = False, num_workers = 2)

# Class label equivalence
classes = ('glass', 'paper', 'cardboard', 'plastic', 'metal', 'trash')

# data_loaders = {"train": trainloader, "val": validloader}
# data_lengths = {"train": len(train_datasets), "val": len(valid_datasets)}

In [0]:
# # Define the network
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(3, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 93 * 125, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 6)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(-1, 16 * 93 * 125)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

# network = Net()



scale = 0.0625
# Define the network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, math.floor(96 * scale), (11, 265), stride = 4, padding = 2)
        self.pool = nn.MaxPool2d(3, stride = 2, padding = 0)
        self.conv2 = nn.Conv2d(math.floor(96 * scale), math.floor(256 * scale), 5, stride = 1, padding = 2)
        self.conv3 = nn.Conv2d(math.floor(256 * scale), math.floor(384 * scale), 3, stride = 1, padding = 1)
        self.conv4 = nn.Conv2d(math.floor(384 * scale), math.floor(384 * scale), 3, stride = 1, padding = 1)
        self.conv5 = nn.Conv2d(math.floor(384 * scale), math.floor(256 * scale), 3, stride = 1, padding = 1)
        self.drop = nn.Dropout(p = 0.5)
        self.thresh = nn.Threshold(0, 1e-6)
        self.fc1 = nn.Linear(math.floor(256 * scale) * 11 * 7, math.floor(4096 * scale))
        self.fc2 = nn.Linear(math.floor(4096 * scale), math.floor(4096 * scale))
        self.fc3 = nn.Linear(math.floor(4096 * scale), 6)
        self.softmax = nn.LogSoftmax()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool(F.relu(self.conv5(x)))
        x = x.view(x.size(0), -1)
        x = self.drop(x)
        x = self.fc1(x)
        x = self.thresh(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.thresh(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x

network = Net()

In [0]:
# Define network's loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(network.parameters(), lr = learnRate, momentum = momentumNum)

In [0]:
# Initialize weight of layers
# takes in a module and applies the specified weight initialization
def weights_init_rule(m):
    classname = m.__class__.__name__
    # for every Linear layer in a model..
    if classname.find('Linear') != -1:
        # get the number of the inputs
        # n = m.in_features
        # y = 1.0/np.sqrt(n)
        # m.weight.data.uniform_(-y, y)
        # m.weight.data.normal_(0.0, y)
        m.bias.data.fill_(0)
        torch.nn.init.kaiming_uniform_(m.weight, 
                              a=0, 
                              mode='fan_in', 
                              nonlinearity='relu')

network.apply(weights_init_rule)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(11, 265), stride=(4, 4), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(24, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (drop): Dropout(p=0.5, inplace=False)
  (thresh): Threshold(threshold=0, value=1e-06)
  (fc1): Linear(in_features=1232, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=6, bias=True)
  (softmax): LogSoftmax()
)

In [0]:
# scheduler = StepLR(optimizer, step_size= 1, gamma= 0.1)

# Training the network
for epoch in range(epochNum):  # loop over the dataset multiple times
    print('Starting epoch: ' + str(epoch))
    
    running_loss = 0.0

    for (inputs, labels) in trainloader:
        # forward pass to get outputs
        outputs = network(inputs)

        # calculate loss between predicted and target
        loss = criterion(outputs, labels)

        # zero the parameter (weight) gradients
        optimizer.zero_grad()

        # backward + optimize only in training
        loss.backward()
        optimizer.step()
        # scheduler.step() # Decay Learning Rate
        
        # update loss
        running_loss += loss.item()

    # epoch_loss = running_loss / data_lengths[phase]
    # print('{} Loss: {}'.format(phase, epoch_loss))
    print(str(running_loss))
        
          
print('Finished Training')

Starting epoch: 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


97.96128833293915
Starting epoch: 1
96.7361900806427
Starting epoch: 2
95.74933552742004
Starting epoch: 3
95.54054272174835
Starting epoch: 4
94.37058913707733
Starting epoch: 5
92.25618696212769
Starting epoch: 6
89.0634970664978
Starting epoch: 7
87.06785309314728
Starting epoch: 8
85.72022426128387
Starting epoch: 9
85.901526927948
Starting epoch: 10
83.98375928401947
Starting epoch: 11
82.43098735809326
Starting epoch: 12
82.77393519878387
Starting epoch: 13
82.24405777454376
Starting epoch: 14
80.45260053873062
Starting epoch: 15
79.53846335411072
Starting epoch: 16
79.36287987232208
Starting epoch: 17
78.88438832759857
Starting epoch: 18
76.65489792823792
Starting epoch: 19
78.69915866851807
Starting epoch: 20
76.35486829280853
Starting epoch: 21
75.85707640647888
Starting epoch: 22
76.96010506153107
Starting epoch: 23
75.12306898832321
Starting epoch: 24
73.48253130912781
Starting epoch: 25
75.012411236763
Starting epoch: 26


In [0]:
# Testing the network on test images
correct = 0
total = 0
with torch.no_grad():
    for (images, labels) in testloader:
        outputs = network(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on test images: %d %%' % (
    100 * correct / total))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on test images: 51 %
